## Predict Diabetic Status

In [1]:
import sys
sys.path.append("../")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from plots import *

now importing: 
3. matplotlib.pyplot as plt



## Load Data

In [2]:
df = pd.read_csv("../data/pima.csv")

In [3]:
df.head()

,NoPregnancies,PlasmaGlucose,DiastolicBP,TricepsSkinThickness,2HourSerumInsulin,BMI,DiabetesPedigreeFunc,Age,IsDiabetic
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
def scale(x):
    mu = np.median(x)
    sd = np.std(x)
    return (x - mu)/sd

In [5]:
cols = ['NoPregnancies', 'PlasmaGlucose', 'DiastolicBP',
 'TricepsSkinThickness', '2HourSerumInsulin', 'BMI',
 'DiabetesPedigreeFunc', 'Age']

for c in cols:
    df[c] = scale(df[c])


In [6]:
df.head()

,NoPregnancies,PlasmaGlucose,DiastolicBP,TricepsSkinThickness,2HourSerumInsulin,BMI,DiabetesPedigreeFunc,Age,IsDiabetic
0,0.890899,0.970212,0.000000,0.752737,-0.264828,0.203071,0.768620,1.786843,1
1,-0.593933,-1.001509,-0.310186,0.376368,-0.264828,-0.685364,-0.064933,0.170176,0
2,1.484832,2.065612,-0.413582,-1.442745,-0.264828,-1.104197,0.904526,0.255263,1
3,-0.593933,-0.876320,-0.310186,0.000000,0.551364,-0.494985,-0.620634,-0.680702,0
4,-0.890899,0.625943,-1.654328,0.752737,1.193898,1.408804,5.785037,0.340351,1


In [7]:
import patsy
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split

In [8]:
y, X = patsy.dmatrices(data=df, formula_like="IsDiabetic ~ NoPregnancies + PlasmaGlucose + DiastolicBP + TricepsSkinThickness + BMI + DiabetesPedigreeFunc + Age")

In [9]:
help(MLPClassifier)

Help on class MLPClassifier in module sklearn.neural_network.multilayer_perceptron:

class MLPClassifier(BaseMultilayerPerceptron, sklearn.base.ClassifierMixin)
 |  Multi-layer Perceptron classifier.
 |  
 |  This model optimizes the log-loss function using LBFGS or stochastic
 |  gradient descent.
 |  
 |  .. versionadded:: 0.18
 |  
 |  Parameters
 |  ----------
 |  hidden_layer_sizes : tuple, length = n_layers - 2, default (100,)
 |      The ith element represents the number of neurons in the ith
 |      hidden layer.
 |  
 |  activation : {'identity', 'logistic', 'tanh', 'relu'}, default 'relu'
 |      Activation function for the hidden layer.
 |  
 |      - 'identity', no-op activation, useful to implement linear bottleneck,
 |        returns f(x) = x
 |  
 |      - 'logistic', the logistic sigmoid function,
 |        returns f(x) = 1 / (1 + exp(-x)).
 |  
 |      - 'tanh', the hyperbolic tan function,
 |        returns f(x) = tanh(x).
 |  
 |      - 'relu', the rectified linear u

In [10]:
model = MLPClassifier(hidden_layer_sizes=(10, 3), activation='relu', alpha=0.000055, 
                      batch_size=200, learning_rate='constant',
                     solver = 'sgd', max_iter = 2000, shuffle = True,
                     verbose = False)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y.ravel(), test_size = 0.25, random_state=123)

In [26]:
model.fit(X=X_train, y = y_train)

MLPClassifier(activation='relu', alpha=5.5e-05, batch_size=200, beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10, 3), learning_rate='constant',
       learning_rate_init=0.001, max_iter=2000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='sgd', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [27]:
model.score(X=X_test, y=y_test)

/home/jonathan/miniconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:108: FutureWarning: np.average currently does not preserve subclasses, but will do so in the future to match the behavior of most other numpy functions such as np.mean. In particular, this means calls which returned a scalar may return a 0-d subclass object instead.
  return np.average(sample_score, weights=sample_weight)


0.78125

In [34]:
X_train.shape

(576, 8)

In [28]:
from keras import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.models import model_from_json

Using TensorFlow backend.


In [75]:
?Dense

In [82]:
model = Sequential()


In [83]:
model.add(Dense(4, input_dim=8, activation='relu', kernel_initializer = 'uniform'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='relu', kernel_initializer = 'uniform'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [84]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])



In [85]:
model.fit(X_train, y_train, epochs=500, verbose=0)


In [86]:
score = model.evaluate(X_test, y_test, batch_size=128)

192/192 [==============================] - 0s 603us/step


In [87]:
score

[0.54993095000584924, 0.78645833333333337]